In [1]:
import SimulateData

In [2]:
config = {
    "signatures_file_path":     'cosmic_signatures/COSMIC_v3.4_SBS_GRCh37.txt',
    "signatures_to_extract":    ['SBS4', 'SBS6', 'SBS7a','SBS9','SBS18'],
    "n_samples":                10,
    "save_dir":                 'simulated_data',
    'sample_signature_distribution': {
        'distribution':             'uniform',
        'min':                      0.5,
        'max':                      2,
        'use_sign_active_prob':     True,
        'sign_active_prob':         0.4,    # Only used if use_sign_active_prob is True
        'n_sign_active':            2       # Only used if use_sign_active_prob is False
    },
    'noise_distribution': {
        'distribution':             'poisson',
        'avg_perc':                 0.05,
    },
    'counts_distribution': {
        'distribution':             'logscale',
        'min':                      1000,
        'max':                      50000,
    }
}

simulated_data, data_file, config_file = SimulateData.simulate_data(config)

Sucessfully saved simulated data in simulated_data/data_v2_GRCh37_4_6_7a_9_18.csv
Sucessfully saved meta-data in simulated_data/config_v2.json

           0    1    2     3    4   5     6     7    8    9
Type                                                       
A[C>A]A  161  213  114  3122  167  67  3057  2027  899  124
A[C>A]C  181  152   96  2338  162  58  2400  1888  719   90
A[C>A]G  139  147   88  2028  143  47  2150  1505  579   92
A[C>A]T  173  171  106  2476  158  58  2496  1839  727  103
A[C>G]A  140  157   99  2031  163  59  2109  1508  627   90


In [4]:
config = {
    "signatures_file_path":     'cosmic_signatures/COSMIC_v3.4_SBS_GRCh37.txt',
    "signatures_to_extract":    ['SBS4', 'SBS6', 'SBS7a','SBS9','SBS18'], # TODO: adjust to be new signatures
    "n_samples":                100,
    "save_dir":                 'simulated_data/experiment_noise_1',
    'sample_signature_distribution': {
        'distribution':             'uniform',
        'min':                      0.5,
        'max':                      2,
        'use_sign_active_prob':     True,
        'sign_active_prob':         0.4,    # Only used if use_sign_active_prob is True
        'n_sign_active':            2       # Only used if use_sign_active_prob is False
    },
    'noise_distribution': {
        'distribution':             'poisson',
        'avg_perc':                 0.05,
    },
    'counts_distribution': {
        'distribution':             'logscale',
        'min':                      1000,
        'max':                      50000,
    }
}

# loop over all the variables to change
noise_avg_list = [0.02, 0.04, 0.08]
mutation_count_list = [(500, 2000), (2000, 5000), (5000, 8000)]
files = []
for avg_perc in noise_avg_list:
    config['counts_distribution']['avg_perc'] = avg_perc
    for mutation_count in mutation_count_list:
        config['noise_distribution']['min'] = mutation_count[0]
        config['noise_distribution']['max'] = mutation_count[1]
        files.append(SimulateData.simulate_data(config, print=False)[1])

print(files)

config_v1.json	data_v1_GRCh37_4_6_7a_9_18.csv
config_v2.json	data_v2_GRCh37_4_6_7a_9_18.csv
